# Char-RNN

In [1]:
from itertools import chain
from keras.models import Sequential

Using TensorFlow backend.


## Load

In [2]:
%store -r descs
len(descs)

171146

## Preprocessing

In [12]:
%time chars = list(set(chain.from_iterable(desc for desc in descs)))
len(chars)

CPU times: user 280 ms, sys: 0 ns, total: 280 ms
Wall time: 278 ms


97

In [15]:
ord('\x07')

7

In [33]:
chr(31)

'\x1f'

In [36]:
chr(31)

'\x1f'

In [35]:
for i in range(32):
    print(repr(chr(i)))

'\x00'
'\x01'
'\x02'
'\x03'
'\x04'
'\x05'
'\x06'
'\x07'
'\x08'
'\t'
'\n'
'\x0b'
'\x0c'
'\r'
'\x0e'
'\x0f'
'\x10'
'\x11'
'\x12'
'\x13'
'\x14'
'\x15'
'\x16'
'\x17'
'\x18'
'\x19'
'\x1a'
'\x1b'
'\x1c'
'\x1d'
'\x1e'
'\x1f'


In [13]:
chars

['\x00',
 'w',
 '>',
 '_',
 '=',
 'P',
 ']',
 '!',
 'A',
 'E',
 'm',
 'u',
 'e',
 'W',
 'Y',
 '+',
 '9',
 'J',
 'o',
 'C',
 '&',
 '5',
 '6',
 '(',
 'O',
 '?',
 'Q',
 'N',
 'c',
 'G',
 'D',
 'X',
 '.',
 '{',
 't',
 '"',
 '1',
 'H',
 '/',
 '-',
 'V',
 'k',
 'h',
 '`',
 'F',
 '#',
 'x',
 '}',
 'g',
 '|',
 'S',
 '\\',
 '7',
 '@',
 'v',
 '3',
 'i',
 "'",
 'd',
 '\x07',
 '$',
 'I',
 'K',
 '8',
 '~',
 ':',
 '4',
 'y',
 '0',
 'B',
 'a',
 'f',
 ';',
 '[',
 'n',
 '<',
 'T',
 's',
 ')',
 '*',
 'b',
 'z',
 'r',
 'q',
 'R',
 'U',
 'j',
 'p',
 '%',
 'l',
 '^',
 '2',
 'Z',
 ',',
 ' ',
 'M',
 'L']